# The problem: turn .json file into a usable .csv file

`json_sample.json` = 10 record sample of research output Pure API response

This is tricksy, as it involves flattening the nested structure of JSON into a CSV. Near certain that to get what I'm after I'll have to code it, anyway, let's see what the AI landscape reckons.

## What's there out of the box? Ask AI

tl;dr: see above.

### GitHub Copilot: `csv` and `json` built-in modules

Code supplied by GitHub Copilot upon prompt "is there a library to turn json into csv?"

In [ ]:
import csv
import json

def json_to_csv(json_file, csv_file):
    with open(json_file, 'r') as j:
        data = json.load(j)  # load json data

    # open a file for writing
    with open(csv_file, 'w') as c:
        writer = csv.writer(c)

        # write the headers
        writer.writerow(data[0].keys())

        # write the data
        for item in data:
            writer.writerow(item.values())

# usage
json_to_csv('json_sample.json', 'sample_output_gitHub.csv')

Runs into JSONDecodeError. Files are in utf-8.
`open()`: if encoding is not specified the encoding used is platform-dependent: `locale.getencoding()` is called to get the current locale encoding. What is that then?

In [ ]:
import locale

locale.getencoding()

Aha, that ain't what we want, hence the decoding error. Pretty easily fixed by just telling `open()` which encoding to use:

In [ ]:
def json_to_csv(json_file, csv_file):
    with open(json_file, 'r', encoding='utf-8') as j:
        data = json.load(j)  

    with open(csv_file, 'w', encoding='utf-8') as c:
        writer = csv.writer(c)

        writer.writerow(data[0].keys())

        for item in data:
            writer.writerow(item.values())

json_to_csv('json_sample.json', 'sample_output_gitHub.csv')

`writer.writerow(data[0].keys())` Expects an array at top level, this file doesn't have that, hence the `KeyError`.

OK, might be better to have a bit of an overview of that `.json` file in the first place. How to get that? Detour to [Problem: Get an overview of a JSON file](#overviewJSON)

Anyway, this is jucky for two reasons:
1. headers are extracted from the first record, so if the records don't have exactly the same this is going to be messy, and, you guessed it, the data I work with is 99,9% like that
2. It ain't working with a root of object type and because of 1. I can't be bothered to even think about how to fix this.

Should say that I reran GitHub Copilot with the ammended prompt used below and got pretty much the same answer from it as from Microsoft Copilot.

Other suggestions?

### ChatGPT: also `csv` and `json` built-in modules

Prompt: Write a script to turn a JSON document into a CSV file.

In [ ]:
import json
import csv

def json_to_csv(input_file, output_file):
    with open(input_file, 'r') as json_file:
        data = json.load(json_file)

    # Extract headers from the keys of the first item
    headers = list(data[0].keys())

    with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    input_file = input("Enter the path to the input JSON file: ")
    output_file = input("Enter the path to the output CSV file: ")
    json_to_csv(input_file, output_file)

Mh, same header problem again. But some different code for the writing to file. `DictWriter`, what's that? [https://docs.python.org/3.11/library/csv.html#csv.DictWriter](https://docs.python.org/3.11/library/csv.html#csv.DictWriter):

In [ ]:
class csv.DictWriter(f, fieldnames, restval='', extrasaction='raise', dialect='excel', *args, **kwds)

Create an object which operates like a regular writer but maps dictionaries onto output rows. The `fieldnames` parameter is a `sequence` of keys that identify the order in which values in the dictionary passed to the `writerow()` method are written to file `f`. The optional `restval` parameter specifies the value to be written if the dictionary is missing a key in `fieldnames`. If the dictionary passed to the `writerow()` method contains a key not found in `fieldnames`, the optional `extrasaction` parameter indicates what action to take. If it is set to `'raise'`, the default value, a `ValueError` is raised. If it is set to `'ignore'`, extra values in the dictionary are ignored. Any other optional or keyword arguments are passed to the underlying `writer` instance.

That sounds rather promising, let's keep this in mind!

Overall, though we have the same header extraction problem as before, so not even going to try that one out.

Anyone else?

### Microsoft Copilot: `pandas` or `csv` and `json`

Ah, that `pandas` approach is new, let's have a look:

In [ ]:
import pandas as pd

# Read the JSON data from a file (replace 'jsonfile.json' with your actual JSON file)
with open('json_sample.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

# Write the data to a CSV file (replace 'csvfile.csv' with your desired output file name)
df.to_csv('sample_output_MSCopilot.csv', encoding='utf-8', index=False)

Mh, nothing happening? Running it in PyCharm ran into this error: `ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.` Whatever that is, I'm not convinced this is the way forward anyway, so qhat's the `csv` and `json` solution suggested:

In [ ]:
import json
import csv

# Load the JSON data from a file (replace 'data.json' with your actual JSON file)
with open('data.json') as json_file:
    data = json.load(json_file)

# Extract relevant fields (assuming your JSON structure is a list of dictionaries)
fieldnames = ['pk', 'model', 'codename', 'content_type', 'name']

# Write the data to a CSV file (replace 'data.csv' with your desired output file name)
with open('data.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for item in data:
        writer.writerow(item)

Pretty much what we've seen before. The second comment is interesting though: `Extract relevant fields (assuming your JSON structure is a list of dictionaries)`. That's exactly the type of assumption we can't make. 

Let's change tact: 

### What if I don't have a list of dictionaries data structure in the JSON file?

GitHub Copilot tells me that: "if your JSON file contains a dictionary of dictionaries, you can still convert it to a CSV file. However, the keys of the outer dictionary will be lost in the conversion". Jup, that's why it's tricky in the first place.

ChatGPT says: "If your JSON file contains \[...] nested objects or arrays of arrays, you may need to handle it differently. One common approach is to flatten the nested structure before converting it to a CSV format."

Microsoft Copilot reckons: "If your JSON contains nested objects (not a flat list of dictionaries), you can flatten it before converting to CSV."

OK, well, I knew that the nesting would be hard from the start. Tired of rooting around. 

### How it would work with regular data

If the data was regular the above script would have of course worked.

In [2]:
import json
import csv

def json_to_csv(json_file, csv_file):
    with open(json_file, 'r', encoding='utf-8') as j:
        data = json.load(j)  

    with open(csv_file, 'w', encoding='utf-8') as c:
        writer = csv.writer(c)

        writer.writerow(data[0].keys())

        for item in data:
            writer.writerow(item.values())

json_to_csv('json_regular.json', 'regular_sample_output_gitHub.csv')

In [3]:
import json
import csv

def json_to_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # Extract headers from the keys of the first item
    headers = list(data[0].keys())

    with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)

json_to_csv('json_regular.json', 'regular_sample_output_chatgpt.csv')

In [4]:
import pandas as pd

# Read the JSON data from a file (replace 'jsonfile.json' with your actual JSON file)
with open('json_regular.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

# Write the data to a CSV file (replace 'csvfile.csv' with your desired output file name)
df.to_csv('regular_sample_output_ms.csv', encoding='utf-8', index=False)
df

,firstname,lastname,age,home_town
0,John,Doe,25,London
1,Jane,Doe,22,New York
2,Jean,Dupont,30,Paris
3,Max,Mustermann,28,Berlin
4,Otto,Normalverbraucher,35,Hamburg
5,Jón,Jónsson,40,Rejkjavik
6,Numerius,Negidius,12,Rome
7,Anna,Kowalska,50,Warsaw
8,Vardenis,Pavardenis,26,Vilnius
9,Jens,Jensen,34,Copenhagen


In [5]:
import pandas as pd

# Read the JSON data from a file (replace 'jsonfile.json' with your actual JSON file)
with open('json_rnest.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

# Write the data to a CSV file (replace 'csvfile.csv' with your desired output file name)
df.to_csv('rnest_sample_output_ms.csv', encoding='utf-8', index=False)
df

,name,age,home_town
0,"{'firstname': 'John', 'lastname': 'Doe'}",25,London
1,"{'firstname': 'Jane', 'lastname': 'Doe'}",22,New York
2,"{'firstname': 'Jean', 'lastname': 'Dupont'}",30,Paris
3,"{'firstname': 'Max', 'lastname': 'Mustermann'}",28,Berlin
4,"{'firstname': 'Otto', 'lastname': 'Normalverbr...",35,Hamburg
5,"{'firstname': 'Jón', 'lastname': 'Jónsson'}",40,Rejkjavik
6,"{'firstname': 'Numerius', 'lastname': 'Negidius'}",12,Rome
7,"{'firstname': 'Anna', 'lastname': 'Kowalska'}",50,Warsaw
8,"{'firstname': 'Vardenis', 'lastname': 'Pavarde...",26,Vilnius
9,"{'firstname': 'Jens', 'lastname': 'Jensen'}",34,Copenhagen


### Conclusion

1. Which headers are expected? -> [Problem: Get an overview of a JSON file](#overviewJSON) might help
2. How to handle nesting? Is there a level that should be the "base-level" against which to flatten? How to express nested values in the CSV?
3. Is there a list in the .json file that can be used to make the above work?

## Potential solution

1. Use the `analyse_json.py` script to get paths and extract expected headers from them
2. Define a baseline and target list agains which to flatten. Decided to flatten the sample so each author gets their own line, which means in cases where there is more than one author item information needs to be repeated. So the `root.items` list is the baseline, and `root.items.[].contributors` the target.

Re point 1, removing the `root` element from the output is not great for reusing the output, removed it for this purpose. Also this time I want the paths sorted so things don't end up all over the place in the output, so added a sort in.

In [1]:
import analyse_json2 
import json_to_csv

analyse_json2.run('json_sample.json', 'sample_analysis2.csv', sort=True)
json_to_csv.run('json_sample.json', 'sample_analysis2.csv', 'sample_to_csv.csv', 'root.items', 'root.items.[].contributors')


Done


Input for the `run()` function are the path to the .json file (`'json_sample.json'`), the analysis file (`'sample_analysis2.txt'`), the base line list name (`'root.items'`) and the target list name (`'root.items.[].contributors'`).

In [ ]:
def run(json_file, header_file, base, target, encoding='utf-8'):
    headers, lists = _get_headers_depth(header_file)
    with open(json_file, 'r', encoding=encoding) as file:
        try:
            data = json.load(file)
            result = _traverse(data, 'root', base, target, lists, None)
        except JSONDecodeError:
            print('JSONDecodeError')
    _format_output(result, base, target, headers)

### `_get_headers()` and `_check_intersection()`

First, get all the elements that will be headers in the output, i.e. the paths that lead to actual values rather than an array or object. Also, I want a dictionary of all the list paths as keys and as their values a dictionary with all their subpaths as keys with blank values. This will later be used to ensure each path is visited for each child of those lists. 

Extract the datatype and path from the json analysis and if it encounters a "list" add the path to the, at this stage, list of lists. If the datatype of the path is not in the `exclusions` the path gets added to `headers` else it's just passed over.

Finally the list of lists is turned into dictionary of dictionaries that'll be needed for list traversal.

In [ ]:
def _get_headers(file_path, has_headers=True, path_index=0, dt_index=2, exclusions=('dict', 'list'),
                 encoding='utf-8'):
    headers = []
    lists_list = []
    with open(file_path, 'r', encoding=encoding) as file:
        reader = csv.reader(file)
        for index, row in enumerate(reader):
            if index == 0:
                if has_headers:
                    continue
            datatypes = row[dt_index].split(';')
            path = row[path_index]
            if 'list' in datatypes:
                lists_list.append(path)
            if _check_intersection(datatypes,
                                   exclusions):  # todo: this is not fool proof if say an element can be list or string
                pass
            else:
                headers.append(path)
    lists = {}
    for li in lists_list:
        lists[li] = {}
        for h in headers:
            if li in h:
                lists[li][h] = ''
    return headers, lists


def _check_intersection(list1, list2):
    return len(set(list1) & set(list2)) > 0

### `_traverse()` and `_format_value()`

This is the same basic traversal as with the analysis. The `out` parameter gets passed around, it'll contain the result, at the start there is None. Can't have an empty `dict` as default parameter, hence the if `None` then `{}` assignment.

If traversing a list that's either the baseline or the target it needs different treatment, hence the `is_target` flag.

In [ ]:
def _traverse(root, name: str, base: str, target_name: str, lists_list, out=None):
        if out is None:
        out = {}
    is_target = False
    if name == target_name or name == base:
        is_target = True

If traversing a `dict`/object run through the children. Names for children are constructed the same way as in the analysis script. This time though I need to reuse `name` later on without the child extension, hence split into `name` and `name_child`.

If the traversal of a child comes back with a literal, add that to `out` with `name_child` as key.

If the child traversed is a list, unpack the `ret` dict into the `out` dict. 

Finally increment the counter variable.

In [ ]:
    if isinstance(root, dict):
        i = 0  # counter variable for sibling elements
        for child in root:  # cycle through the children of root element
            name_child = name + "." + child
            ret = _traverse(root[child], name_child, base, target_name, lists_dict, out=out)
            if isinstance(ret, str):
                out[name_child] = ret
            if isinstance(root[child], list):
                out = _unpack(out, ret, is_target)
            i = i + 1
        return out

If traversing a `list`/array grab a deep copy of the dictionary with this lists children as keys. Reason is that not all paths may be present in all children of the list, but I need a return value for each path, so I can later access them via index. If the path isn't present in the child then the return value is an empty string.

Now run through the children. This time we don't want to pass the same `out` variable as before as the result will be different for each child and they'd overwrite each other, so each child get's its own `_out`. 

If the child traversal brings back a literal it means we're dealing with a list like `["a", "b", "c"]` and it just get's dumped back into an semicolon separated list of literal values under the respective path name. Semicolon as I don't want them to get mixed up with the comma separation.

Otherwise, we need to incorporate the traversal result into the `list_headers` dict.

Finally increment the counter.

In [ ]:
    elif isinstance(root, list):  
        i = 0  # counter variable for sibling elements
        list_headers = deepcopy(lists_dict[name])
        for child in root:  # cycle through the children of root element
            _out = {}
            name_child = name + '.[]'
            ret = _traverse(child, name_child, base, target_name, lists_dict, out=_out)
            if isinstance(ret, str):
                try:
                    prev_value = out[name]
                    if prev_value != '':
                        out[name] = prev_value + ';' + ret
                except KeyError:
                    out[name] = ret
            else:
                list_headers = _join(list_headers, _out, i, is_target)

            i = i + 1
        return list_headers

If the traversal encounters a literal value (anything but `list` or `dict` basically), format that value into a string and, if it contains a comma double-quote it.

In [ ]:
    else:
        return _format_value(root)

def _format_value(data_to_write):
    if isinstance(data_to_write, str):
        if ',' in data_to_write:
            data_to_write = '"' + data_to_write + '"'
        return data_to_write
    if isinstance(data_to_write, int) or isinstance(data_to_write, float) or isinstance(data_to_write, bool):
        return str(data_to_write)
    if data_to_write is None:
        return ''

### `_unpack()`

This is merges a child `dict` into the parent `dict`, where the keys do not already exist in `parent_dict` as this was a `dict` iteration. Iterate over the keys in `child_dict`. 

`parent_dict` should **never** already contain any of the keys in `child_dict` hence the Error if not.

If the value of the respective key is a `list` the length of the list tells how it should be treated. If it has a length of `1`, just grab that value and dump it as the value for the key. If it's longer than `1`, i.e. this is either the base list or target list, dump the list as it is as the value. Empty lists should not happen hence the error if it does.

`child_dict = {"key": ["bli;bla;blubb"]}` -> literals that got concatenate, i.e. that will not be expanded into their own rows

`child_dict = {"key": [bli, bla, blubb]}` -> literals that will get expanded

In [ ]:
def unpack(parent_dict, child_dict):
    for key in child_dict.keys():
        if key not in parent_dict.keys():
            if isinstance(cv := child_dict[key], list):
                if len(cv) > 1:
                    parent_dict[key] = cv
                elif len(cv) == 1:
                    parent_dict[key] = cv[0]
                else:
                    raise ValueError('Empty list')
            else:
                parent_dict[key] = child_dict[key]
        else:
            raise ValueError('Key already exists')
    return parent_dict

### `_join()`

This one is merging list child values into the list parent values. Keys will already exist in `parent_dict` as it from the start contained all possible keys in the list.

Iterate over the `parent_dict` keys and try the same key on the `child_dict`. If the path didn't exist in the respective child this will throw a `KeyError` in which case the child's value for this key is an empty string. Else the child's value is whatever the `child_dict` has.

If this is the first child (`index == 0`), no need for any fuss, just dump the child value into the respective key in the `parent_dict`. If this is a subsequent child however a decision needs to be made as to how to append the value. 

First, let's grab the `parent_value`, as this will change before it get's written back to the `parent_dict` and that can't be done with it in situ.

Are we dealing with either the base or the target level? If so, the `parent_value` should be a `list` already, if not there's something off. Append the `child_value` and write the new `parent_value` back to the `parent_dict`.

If we're not dealing with base or target level then no appending, but concatenating. Child values at this stage are `lists`, so we need to grab the string from it. There should be only one entry in the list anyway. Concatenate the `child_value`, `;` and `parent_value` and write the lot back to the `parent_dict`.

In [ ]:
def _join(parent_dict, child_dict, index, is_target_level=False):
    for key in parent_dict.keys():
        try:
            child_value = child_dict[key]
        except KeyError:
            child_value = ''
        if index == 0:
            parent_dict[key] = [child_value]
        else:
            parent_value = parent_dict[key]
            if is_target_level:
                if not isinstance(parent_value, list):
                    raise ValueError('Not a list')
                else:
                    parent_value.append(child_value)
                    parent_dict[key] = parent_value
            else:
                if not isinstance(parent_value, str):
                    parent_value = parent_value[0]
                else:
                    raise ValueError('Not a string')
                new_value = parent_value + ';' + child_value
                parent_dict[key] = [new_value]
    return parent_dict

### `_format_output()` and `_output()`

First write the headers to file.

Then we need to separate the headers into those that will always contain the same values (`constants`) as they are above the base level, those that are at base level (`base_fields`) and those that are at target level (`target_fields`). We'll do this by evaluating the path names in the `headers` list. The base and target list paths won't appear in the `headers` though as they are paths for `lists` not for literals, so let's append the list marker to them so they can be matched to values in `headers`.

Now iterate over the values in `headers`. The order of the if statements matters here. Fishing out the constants is easy as they are the only ones that at this stage will have literal values. Then the `target_fields` as they will also match for having `base_list` in their path and we need to get them out of the way before that can happen. Finally all else should end up in the `base_fields` list, if not give me an error.

In [ ]:
def _format_output(result, base, target, headers):
    output(None, headers, head=True)
    constants = []
    base_fields = []
    target_fields = []
    base_list = base + '.[]'
    target_list = target + '.[]'
    for h in headers:
        value = result[h]
        if not isinstance(value, list):
            constants.append(h)
            continue
        if target_list in h:
            target_fields.append(h)
            continue
        if base_list in h:
            base_fields.append(h)
            continue
        raise ValueError('Field not a constant, in base or target')

Now, we need to know how long our base list is, easy, we just grab a value from `base_fields`, retrieve the value associated with that key from results and see how many values are in that array. That length is the same for all keys in `base_fields`, the `join()` function made sure of that by assigning a value for each path for each child. 

Now set up counter and the `dict` that will contain the data for an individual row.

First populate this with the constant values. That never needs repeating as they never change again, the value is the same for each row. 

Then we iterate through keys in `base_fields` and add the value from the respective array slot to `row`.

Finally for the `target_fields`. First we check if there is an array of values to deal with, as will be the case if there is one than more author). If so we need the length of that array (`target_length`) and a counter (`target_counter`). Then we iterate over the `target_fields` and retrieve the respective value as the array slot annd add it to `row`. 

Now the row is ready for `_output()`. Increment the counter and process the next array slot. This means the `target_fields` values change but everything else stays the same. 

If there isn't a list to deal with, then just iterate over `target_fields`, extract the values and output the `row`.

Then on to the next `base_field` and repeat the spiel until all have been processed. 

In [ ]:

    base_length = len(result[base_fields[0]])
    base_counter = 0
    row = {}
    for constant in constants:
        row[constant] = result[constant]
    while base_counter < base_length:
        for base_field in base_fields:
            row[base_field] = result[base_field][base_counter]
        if isinstance(x := result[target_fields[0]][base_counter], list):
            target_length = len(x)
            target_counter = 0
            while target_counter < target_length:
                for target_field in target_fields:
                    row[target_field] = result[target_field][base_counter][target_counter]
                output(row, headers)
                target_counter += 1
        else:
            for target_field in target_fields:
                row[target_field] = result[target_field][base_counter]
            output(row, headers)
        base_counter += 1

This one just handles the writing to file of rows, using the `csv.DictWriter`. One can feed this a list of headers and then dictionaries of those headers as keys and it then concatenates the values into the right order of comma delimited values.

In [ ]:
def _output(_data, headers, head=False):
    with open('resources/sample_out.txt', 'a', encoding='utf-8', newline='') as out_file:
        writer = csv.DictWriter(out_file, fieldnames=headers)
        if head:
            writer.writeheader()
        else:
            writer.writerow(_data)

<a id="overviewJSON"></a>
## Problem: Get an overview of a JSON file

I want to know:
- what **paths** are in the file,
- what their **value data type** is and
- **how often** they occur.

The tools I found and tried all didn't allow me to get the data I wanted. 

- [jq](https://jqlang.github.io/jq/), a command-line JSON processor, does have a function(s) to output paths, but they aren't quite what I'm after. Maybe I could have fiddled with this and made it work, but then I may as well just write it in Python in the first place.
- Notepad++ extensions I tried also didn't do the right thing, they have tree views to get a better overview of the file, but don't do any of the stats things I want

By this point I'd spent quite some time trying to find an existing solution and ran out of things to google, so decided to write it myself. See `analyse_json.py`.

Here's what it does with the the `json_sample.json` file.

In [1]:
import analyse_json

analyse_json.run('json_sample.json', 'sample_analysis.csv')

Done


In [ ]:
import analyse_json_gui

analyse_json_gui.run()

In [ ]:
import analyse_json

analyse_json.run('json_sample.json', 'sample_analysis.csv')

import pandas

analysis = pandas.read_csv('sample_analysis.csv')
pandas.set_option('display.max_rows', 200)
pandas.set_option('display.colheader_justify','left')
analysis = analysis.style.set_properties(**{'text-align': 'left', 'font-family': 'Monospace'})
analysis

### `run()`

The `run()` function (see above) is the only public function and called to start the script. Provide the path to the input and output file as arguments. If needed the encoding scheme needed can be set as well; the default encoding scheme is UTF8.

It sets instantiates the `tags` variable which will hold the encountered paths, they counts and data types.
It checks that `input_file` and `output_file` file paths are actually provided and calls `set_file()` if one is missing. For `input_file` with the `mandatory` parameter set to `True`. For `input_file` it also checks that the file actually exists and if not calls `_set_file()` to rectify this. 

In [7]:
def run(input_file: str = None, output_file: str = None, encoding='utf-8'):
    tags = {}
    if input_file is None:
        input_file = _set_file('file to analyse', True)
    if not os.path.isfile(input_file):
        print('Provided input file does not exist.')
        input_file = _set_file('file to analyse', True)
    if output_file is None:
        output_file = _set_file('output file')

If both `input_file` and `output_file` are not `None` it tries to open the `input_file` and load it as json. Should this run into a `JSONDecodeError` it is caught and the script execution stopped. At this point there should be no possibility of `input_file` or `output_file` still being `None`, but just in case there is a hole in my logic...

`JSONDecodeError` can happen if the file is not valid JSON or if there are characters in the file that can't be decoded with the provided encoding scheme.

In [ ]:
    if input_file is not None and output_file is not None:
        try:
            with open(input_file, 'r', encoding=encoding) as file:
                data = json.load(file)
        except json.JSONDecodeError as e:
            print(
                "Input file could not be decoded. Could be because it's not a valid JSON file or it contains characters that cannot be decoded with the provided encoding scheme. Exiting script. JSONDecodeError: " + str(
                    e))
            raise StopExecution

Next the `traverse()` function is called to run through the actual analysis. `'root'` is provided as the value for the `name` paramenter as it needs a name of some sort.

In [ ]:
        _traverse(data, 'root', tags)

Finally, the `output()` function is called to output the data now held in the `tags` variable. Enclosed it in a `try/except` block to catch any issue with writing to the provided path and rather than running into trouble the script dumps the value of `tags` into a cache file at root level and gracefully exits. 

Right at the end is the graceful exit if `input_file` or `output_file` were still `None`.

In [ ]:
        try:
            _output(tags, output_file, encoding)
        except IOError as e:
            print("Output file could not be written. Dumping to cache and exiting script. IOError: " + str(e))
            with open('cache.txt', 'w', encoding=encoding) as cache:
                cache.write(str(tags))
            raise StopExecution
    else:
    print("Input file or output file missing. Exiting script.")
    raise StopExecution   

### `traverse()`

The main function is `traverse()`. It recursively traverses the entire document and fishes out all paths it encounters, triggers recording their value data type and counting. Its paramenter are the element is is evaluating (`root`), its name/path (`name`) and the `tags` dictionary containing all previously encountered paths, they counts and data types. 

It only needs to act recursively is the passed in element is either a `list` or `dict`. In this case it sets up a counter for sibling elements and then loops over the children in the respective `list` or `dict`. 

For `root`s that are `dict` the name of the child element preceded by `.` is appended to `name` and the `traverse()` function called with the child as the `root` parameter value.

For `root`s that are `list` `.[]` is appended to the name as there is no key to use and the `traverse()` function called with the child as the `root` parameter value.

Finally, the sibling counter is incremented.

If the current child is a sibling, the script first removes the previously appended `name` part.

In [ ]:
def _traverse(root, name: str, tags: dict) -> None:
    _record_datatype(name, tags, type(root))
    _count_tag(name, tags)
    if isinstance(root, list) | isinstance(root, dict):  # check if passed element is a list or dictionary
        i = 0                                            # counter variable for sibling elements
        for child in root:                               # cycle through the children of root element
            if i > 0:                                    # i > 0 means this is sibling element and last name element needs removing
                try:
                    name = name[:name.rindex(".")]
                except ValueError:
                    name = name
                    print('ValueError: ' + name)

            if isinstance(root, list):
                name = name + '.[]'
                _traverse(child, name, tags)

            else:
                name = name + "." + child
                _traverse(root[child], name, tags)
            i = i + 1

### `_count_tag()`, `_record_datatype()` and `setup()`

`_count_tag()` takes an element name/path and the `tags` dictionary as paramenters. It tries to grab a previous count value for the respective path and increments it. If there is no previous count because this path hasn't been previously encountered it calls `_setup` to instantiate the respective dictionary holding count and data types within itself.

In [ ]:
def _count_tag(tag_name: str, tags: dict) -> None:
    try:
        count = tags[tag_name]['count']
        tags[tag_name]['count'] = count + 1
    except KeyError:
        tags[tag_name] = _setup(count=1)

`_record_datatype()` also takes an element name/path and the `tags` dictionary as paramenters, plus the element's [`Type`](https://docs.python.org/3.11/library/stdtypes.html#type-objects). First it turns the `Type` objects string representation into something nicer to read. It then also tries to grab any previously deposited data type values and, if the respective one isn't already in there, append it to them. If there are no previous values, because the path hasn't been previously encountered it calls `_setup` to instantiate the respective dictionary holding count and data types within itself.

In [ ]:
def _record_datatype(tag_name, tags: dict, datatype) -> None:
    datatype = str(datatype).replace("<class '", "").replace("'>", "")
    try:
        datatypes = tags[tag_name]['datatypes']
        if datatype not in datatypes:
            datatypes.append(datatype)
    except KeyError:
        tags[tag_name] = _setup(datatype=datatype)

`_setup()` instantiates the dictionary holding count and data types for a path within the `tags` dictionary. It can be called either with an initial count value or an initial data type value. An initial `count` value is always 0, but `datatypes` and either be an empty `list` (if the call is coming from `_count_tag()` or have an initial value (if the call is coming from `_record_dataype()`.

In [ ]:
def _setup(count=0, datatype=None):
    d = {'count': count}
    if datatype is not None:
        d['datatypes'] = [datatype]
    else:
        d['datatypes'] = []
    return d

### `_output()` and `_format_list_content`

`_output()` handles writing the `tags` dictionary to a file in CSV. Once the file is opened it writes the header line and then iterates through `tags`. 

For each entry in `tags` it retrieves the path, i.e. the key to the inner dictionary holding the count and datatypes. It only keeps `'root.'` for the actual root element, else everything would start in 'root' and I just find that ugly. 

Then it grabs the `list` holding the data types gets it formatted.

Finally it concatenates path, count, data types into a comma-delimited output string and writes it to file.

Just added the "Done" print as otherwise there is no indication in the notebook of the script having finished.

In [ ]:
def _output(tag_set: dict, out_file_path, encoding) -> None:
    with open(out_file_path, 'w', encoding=encoding) as out_file:
        out_file.write("tag,count,datatypes\n")
        for tag in tag_set:
            tag_out = 'root'
            if tag != tag_out:
                tag_out = tag.replace('root.', '')
            datatypes = _format_list_content(tag_set[tag]['datatypes'])
            out_file.write(tag_out + "," + str(tag_set[tag]['count']) + ',' + datatypes + "\n")
    print('Done')

`_format_list_content()` just makes sure that the values from the list are properly separated and there are no unnecessary separators at the start and end. It treats all elements but the last the same.

In [ ]:
def _format_list_content(list_in):
    out = ''
    for i, l in enumerate(list_in):
        out = out + l
        if i < len(list_in) - 1:
            out = out + ';'
    return out

### `_set_file()`

`_set_file()` asks for user input to obtain a file path. Its parameters are a text snippet to output to let the user know what their entering a value for (`text`) and whether the file must already exist (`mandatory`). 

`mandatory` is `False` by default, but if it is set to `True` the function then checks that the file exists and if not calls itself again to obtain a file path to an existing file. This can technically become an endless loop, but I just couldn't be bothered to do something about that.

In [ ]:
def _set_file(text, mandatory=False):
    file_path = input(f"path to {text}: ")
    if mandatory:
        if os.path.isfile(file_path):
            return file_path
        else:
            _set_file(f'Provided file does not exist. Please provide a valid path to the {text}: ', True)
    else:
        return file_path

### `StopExecution`

This only exists because I don't want Jupyter to display the entire stack trace when stopping the script, it's ugly and not neccessary in this context.

### Things to improve

Currently the output in is the order that it's added to the `tags` dictionary. Maybe sort it alphabetically instead? Or by count? Or something else?

### Got a GUI?

Yup, try `analyse_json_gui.py` (not in the notebook though, wont' work). Dependency is `easygui`. Why? Because I can.

In [ ]:
pip install easygui

In [ ]:
import csv
import json


def json_to_csv(json_file, csv_file):
    with open(json_file, 'r', encoding='utf-8') as j:
        data = json.load(j)  # load json data

    # open a file for writing
    with open(csv_file, 'w', encoding='utf-8') as c:
        writer = csv.writer(c)

        # write the headers
        writer.writerow(data[0].keys())

        # write the data
        for item in data:
            writer.writerow(item.values())


# usage
json_to_csv('G:\\My Drive\\python\\util_scripts\\resources\\wos\\json_20240321.json', 'output.csv')